In [1]:
import numpy as np
import pandas as pd

np.random.seed(12345) # for learning purposes
pd.options.display.max_rows = 10

# 10.1 GroupBy mechanics

Each grouping keys can take many forms, and the keys do not have to be all of the same type:
- A list or array of values that is the same length as the axis being grouped
- A value indicating column name in a DataFrame
- A dict or Series giving a correspondence between the values on the axis being grouped and the group names
- A function to be invoked on the axis index or the individual labels in the index

In [2]:
# create data
df = pd.DataFrame({'key1' : ['a' ,'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [3]:
# suppose you wanted to compute the mean of the data1 column using the labels from key1
grouped = df['data1'].groupby(df['key1']) # it returns a 'GroupBy' object

# this object has all of the information needed to then apply some operation to each of the groups
print('GroupBy object:', grouped)
print('---------------------------')

# to compute group means, call the GroupBy's mean() method
print(grouped.mean())

GroupBy object: <pandas.core.groupby.generic.SeriesGroupBy object at 0x000002090012C670>
---------------------------
key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64


In [4]:
# to group data using multiple keys, pass in a list 
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

means

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

In [5]:
# the grouped data can then be unstacked
means.unstack()

key2,one,two
key1,,
a,0.880536,0.478943
b,-0.519439,-0.555730


In [6]:
# the group keys can be any arrays of the right length
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

In [7]:
# frequently the grouping information is found in the same DataFrame as the data you want to work on
# in that case, you can pass the column names as the group keys

print(df.groupby('key1').mean())
print('---------------------------------')
print(df.groupby(['key1', 'key2']).mean())

         data1     data2
key1                    
a     0.746672  0.910916
b    -0.537585  0.525384
---------------------------------
              data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023


In [8]:
# a generally useful GroupBy method is size(), which returns a Series containing group sizes
df.groupby(['key1', 'key2']).size() 

# take note that any missing values in a group key will be excluded from the result

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 10.1.1 Iterating Over Groups

In [9]:
# the GroupBy object supports iteration
# generating a sequence of 2-tuples containing the group name along with the chunk of data

for name, group in df.groupby('key1'):
    print('group name =', name)
    print('-----------------------------------')
    print(group)
    print('*')

group name = a
-----------------------------------
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
*
group name = b
-----------------------------------
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023
*


In [10]:
# in the case of multiple keys, the first element in the tuple will be a tuple of key values

for (k1, k2), group in df.groupby(['key1', 'key2']):
    print('group name =',(k1, k2))
    print('------------------------------------')
    print(group)
    print('*')

group name = ('a', 'one')
------------------------------------
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435
*
group name = ('a', 'two')
------------------------------------
  key1 key2     data1     data2
1    a  two  0.478943  0.092908
*
group name = ('b', 'one')
------------------------------------
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
*
group name = ('b', 'two')
------------------------------------
  key1 key2    data1     data2
3    b  two -0.55573  0.769023
*


In [11]:
# a recipe you may find useful is computing a dict of the data pieces as a one-liner

pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023


In [12]:
# by default, groupby() groups on axis=0, but you can group on any other axes

# for example, we could group the columns of our example df here by dtype like so
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [13]:
grouped = df.groupby(df.dtypes, axis=1)

# we can print out the groups like so
for dtype, group in grouped:
    print('group name =', dtype)
    print('-----------------------------------')
    print(group)
    print('*')

group name = float64
-----------------------------------
      data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435
*
group name = object
-----------------------------------
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one
*


## 10.1.2 Selecting a Column or Subset of Columns

In [14]:
# indexing a groupby object created from a DataFrame with a column name or array of column names 
# has the effect of column subsetting for aggregation

# this means that
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
# are syntactic sugar for 
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

# especially for large datasets, it may be desirable to aggregate only a few columns
# for example, in the preceding dataset:
# to compute means for just the 'data2' column and get the result as a DataFrame, we could write
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

In [15]:
# the object returned by indexing operation is:

# a 'grouped DataFrame' if a list or array is passed, or
print('grouped DataFrame:', df.groupby(['key1', 'key2'])[['data2']])

# a 'grouped Series' if only a single column name is passed as a scalar
print('grouped Series:   ', df.groupby(['key1', 'key2'])['data2'])

grouped DataFrame: <pandas.core.groupby.generic.DataFrameGroupBy object at 0x00000209029E7190>
grouped Series:    <pandas.core.groupby.generic.SeriesGroupBy object at 0x00000209029E7070>


## 10.1.3 Grouping with Dicts and Series

In [16]:
# grouping information may exist in a form other than an array

# create data
people = pd.DataFrame(np.random.randn(5, 5), 
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan

people

,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [17]:
# suppose we have a group correspondence for the columns and want to sum together the column by group
mapping = {'a' : 'red', 'b' : 'red', 'c' : 'blue', 'd' : 'blue', 'e' : 'red', 'f' : 'orange'}

# now, you could construct an array from this dict to pass to groupby()
# but instead we can just pass the dict (key 'f' is intentionally included to highlight that unused grouping keys are OK)
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.503905,1.063885
Steve,1.297183,-1.553778
Wes,-1.021228,-1.116829
Jim,0.524712,1.770545
Travis,-4.230992,-2.405455


In [18]:
# the same functionality holds for Series, which can be viewed as a fixed-size mapping
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [19]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## 10.1.4 Grouping with Functions

In [20]:
# using Python functions is a more generic way of defining a group mapping
# any function passed as a group key will be called once per index value, with the return values being used as the group names

# consider the previous example
# suppose you wanted to group by the length of the names
# while you could compute an array of string lengths, it's simpler to just pass the 'len' function
people.groupby(len).sum()

,a,b,c,d,e
3,0.591569,-0.993608,0.798764,-0.791374,2.119639
5,0.886429,-2.001637,-0.371843,1.669025,-0.438570
6,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [21]:
# mixing functions with arrays, dict, or Series is not a problem as everything gets converted to arrays internally
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.539741 -1.296221  0.274992 -1.021228 -0.577087
  two  0.124121  0.302614  0.523772  0.000940  1.343810
5 one  0.886429 -2.001637 -0.371843  1.669025 -0.438570
6 two -0.713544 -0.831154 -2.370232 -1.860761 -0.860757

## 10.1.5 Grouping by Index Levels

In [22]:
# a final convenience for hierarchically indexed datasets is 
# the ability to aggregate using one of the levels of an axis index

# create data
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

# modify some value to create highlight points
hier_df.iloc[1, 1] = np.nan 
hier_df.iloc[2, 2:4] = np.nan

# display data
hier_df 

cty          US                            JP          
tenor         1         3         5         1         3
0      0.560145 -1.265934  0.119827 -1.063512  0.332883
1     -2.359419       NaN -1.541996 -0.970736 -1.307030
2      0.286350  0.377984       NaN       NaN  1.349742
3      0.069877  0.246674 -0.011862  1.004812  1.327195

In [23]:
# to group by level, pass the level number or name using the 'level' keyword
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,2
2,1,2
3,2,3


# 10.2 Data Aggregation

In [24]:
# Aggregations refer to any data transformation that produces scalar values from arrays

# you can use aggregations of your own devising and additionally call any method 
# that is also defined on the grouped object

# display data
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [25]:
# for example quantile(), while not explicitly implemented for GroupBy, is a Series method, and thus available for use

grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.668413
b   -0.523068
Name: data1, dtype: float64

In [26]:
# to use your own aggregation functions, pass any function that aggregates an array to the aggregate() or agg() method
def peak_to_peak(arr):
    return arr.max() - arr.min()

# remember to  select the columns that should be valid for the aggregate function (['data1', 'data2'] in this case)
grouped[['data1', 'data2']].aggregate(peak_to_peak)

,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


In [27]:
# you may notice that some method like describe() also work, even though they are not aggregations
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.746672  1.109736 -0.204708  0.137118  0.478943  1.222362   
b      2.0 -0.537585  0.025662 -0.555730 -0.546657 -0.537585 -0.528512   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.965781   3.0  0.910916  0.712217  0.092908  0.669671  1.246435   
b    -0.519439   2.0  0.525384  0.344556  0.281746  0.403565  0.525384   

                          
           75%       max  
key1                      
a     1.319920  1.393406  
b     0.647203  0.769023

## 10.2.1 Column-Wise and Multiple Function Application

In [28]:
# create data
tips = pd.read_csv('examples/tips.csv')

# add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [29]:
# grouping 
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']

# note that for descriptive statistics, you can pass the name of the function as a string
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [30]:
# if you pass a list of functions or function names instead
# you get back a DataFrame with column names taken from the function
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [31]:
# you don't have to accept the names that GroupBy gives to the columns
# if you pass a list of (name, function) tuples, the first element of each tuple will be used as the DataFrame column names
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [32]:
# with DataFrame you have more options
# as you can specify a list of functions to apply to all of the columns of different functions per column

# suppose we want to compute the same three statistics for the 'tip_pct' and 'total_bill' column
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'total_bill']].aggregate(functions)

result
# as you can see, the resulting DataFrame has hierarchical columns
# the same as you would get aggregating each column separately 
# and using concat() to glue the results together using the column names as the 'key' argument

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [33]:
# as before, a list of tuples with custom names can be passed
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped[['tip_pct', 'total_bill']].aggregate(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [34]:
# now, suppose you wanted to apply potentially different functions to one or more of the columns
# to do this, pass a dict to agg() that contains a mapping of column names to any of the function specifications listed so far

grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [35]:
# another example
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

# a DataFrame will have hierarchical columns only if multiple functions are applied to at least one column

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## 10.2.2 Returning Aggregated Data without Row Indexes

In [36]:
# in all of the examples up until now, the aggregated data comes back with an index
# since this isn't always desireable, you can disable this behavior by passing as_index=False to groupby()
tips.groupby(['day', 'smoker'], as_index=False).mean()

# you can also acheive the same result by calling reset_index() on the result
# but using as_index=False avoids some unnecessary computations

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 10.3 Apply: General split-apply-combine

In [37]:
# the most general-purpose GroupBy method is apply(), which is the subject of this section
# apply() does the following: 
#         # splits the object being manipulated into pieces
#         # invoke the passed function on each piece
#         # concatenate the pieces together

# returning to the tipping dataset, suppose you want to select the top five 'tip_pct' values by group
# first, write a function that selects the rows with the largest values in a particular column
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column, ascending=True)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [38]:
# now, if we group by 'smoker', and call apply() with this function, we get the following
tips.groupby('smoker').apply(top)

# the 'top' function is called on each row group from the DatFrame
# then the result are glued together using pandas.concat(), labeling the pieces with the group names
# the result therefore has a hierarchical index whose inner level contains index values from the original DataFrame

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [39]:
# if you pass a function to apply() that takes other arguments or keywords, you can pass these after the function

tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [40]:
# you may recall that describe() can be called on a GroupBy object
result = tips.groupby('smoker')['tip_pct'].describe()

result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [41]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
                    ...    
50%    Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
Length: 16, dtype: float64

In [42]:
# inside GroupBy, when you invoke a method like describe(), it is actually just a short cut for
f = lambda x: x.describe()
tips.groupby('smoker')['tip_pct'].apply(f).unstack()

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


## 10.3.1 Suppressing the Group Keys

In [43]:
# in the preceding examples, you see that the resulting object has a hierarchical index 
# formed from the group keys along with the indexes of each piece of the original object
# you can disable this by passing group_keys=False to groupby()

tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


## 10.3.2 Quantile and Bucket Analysis

In [44]:
# as you may recall, pandas has some tools, in particular cut() and qcut()
# for slicing data up into buckets with bins  of your choosing or by sample quantiles
# combining these functions with groupby() makes it convenient to perform 'bucket' or 'quantile' analysis on a dataset

# create data
frame = pd.DataFrame({'data1' : np.random.randn(1000),
                      'data2' : np.random.randn(1000)})

quartiles = pd.cut(frame.data1, 4) # cut into 4 equally spaced part
quartiles[:10]

0     (-1.23, 0.489]
1    (-2.956, -1.23]
2     (-1.23, 0.489]
3     (0.489, 2.208]
4     (-1.23, 0.489]
5     (0.489, 2.208]
6     (-1.23, 0.489]
7     (-1.23, 0.489]
8     (0.489, 2.208]
9     (0.489, 2.208]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

In [45]:
# the 'Categorical' object returned by cut() can be passed directly into groupby()

# so we could compute a set of statistics for the data2 column like so
def get_stats(group):
    return {'min' : group.min(), 'max' : group.max(),
            'count' : group.count(), 'mean' : group.mean()}

grouped = frame['data2'].groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.956, -1.23]",-3.399312,1.670835,95.0,-0.039521
"(-1.23, 0.489]",-2.989741,3.260383,598.0,-0.002051
"(0.489, 2.208]",-3.745356,2.954439,297.0,0.081822
"(2.208, 3.928]",-1.929776,1.765640,10.0,0.024750


In [46]:
# to compute equal-size buckets based on sample quantiles, use qcut()

grouping = pd.qcut(frame.data1, 10, labels=False) # labels=False to get the quantile numbers
grouped = frame['data2'].groupby(grouping) 
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-3.399312,1.670835,100.0,-0.049902
1,-1.950098,2.628441,100.0,0.030989
2,-2.925113,2.527939,100.0,-0.067179
3,-2.315555,3.260383,100.0,0.065713
4,-2.047939,2.074345,100.0,-0.111653
5,-2.989741,2.184810,100.0,0.052130
6,-2.223506,2.458842,100.0,-0.021489
7,-3.056990,2.954439,100.0,-0.026459
8,-3.745356,2.735527,100.0,0.103406


## 10.3.3 Example: Filling Missing Values with Group-Specific Values

In [47]:
# when cleaning up missing data, in some cases you will replace data observations using dropna()
# but in others you may want to impute (fill in) the NA values using a fixed value or some value derived from the data
# fillna() is the right tool to use

# create data
s = pd.Series(np.random.randn(6))
s[::2] = np.nan

s

0         NaN
1   -0.125921
2         NaN
3   -0.884475
4         NaN
5    0.227290
dtype: float64

In [48]:
# here we replace missing value with the mean
s.fillna(s.mean())

0   -0.261035
1   -0.125921
2   -0.261035
3   -0.884475
4   -0.261035
5    0.227290
dtype: float64

In [49]:
# suppose you need the fill value to vary by group
# one way to do this is to group the data and use apply() with a function that calls fillna() on each datachunk

# here is some sample data on US states devided into Eastern and Western regions
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_keys = ['East'] * 4 + ['West'] * 4

data = pd.Series(np.random.randn(8), index=states)

print('pre missing data: ')
print(data)
print('-------------------------------------')
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
print('post missing data: ')
print(data)
print('-------------------------------------')

pre missing data: 
Ohio          0.922264
New York     -2.153545
Vermont      -0.365757
Florida      -0.375842
Oregon        0.329939
Nevada        0.981994
California    1.105913
Idaho        -1.613716
dtype: float64
-------------------------------------
post missing data: 
Ohio          0.922264
New York     -2.153545
Vermont            NaN
Florida      -0.375842
Oregon        0.329939
Nevada             NaN
California    1.105913
Idaho              NaN
dtype: float64
-------------------------------------


In [50]:
data.groupby(group_keys).mean() # compute the group mean 

East   -0.535707
West    0.717926
dtype: float64

In [51]:
# we can fill the NA values using the group mean like so
fill_mean = lambda g : g.fillna(g.mean())
data.groupby(group_keys).apply(fill_mean)

Ohio          0.922264
New York     -2.153545
Vermont      -0.535707
Florida      -0.375842
Oregon        0.329939
Nevada        0.717926
California    1.105913
Idaho         0.717926
dtype: float64

In [52]:
# in another case, you might have predefined fill values in your code that vary by group
# since the groups have a 'name' attribute set internally, we can use that
fill_values = {'East' : 0.5, 'West' : -1}
fill_func = lambda g : g.fillna(fill_values[g.name])
data.groupby(group_keys).apply(fill_func)

Ohio          0.922264
New York     -2.153545
Vermont       0.500000
Florida      -0.375842
Oregon        0.329939
Nevada       -1.000000
California    1.105913
Idaho        -1.000000
dtype: float64

## 10.3.4 Example: Random Sampling and Permutation

In [53]:
# suppose you wanted to draw a random sample (with or without replacement) from a large dataset
# there are a number of ways to perform the "draw"; here we use the sample() method for Series
# to demonstrate, here's a way to construct a deck of English-style playing cards

# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D'] 
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list (range(2, 11)) + ['J', 'Q', 'K']
cards = []
for suit in suits:
    cards.extend([str(num) + suit for num in base_names])

deck = pd.Series(card_val, index=cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
       ..
9D      9
10D    10
JD     10
QD     10
KD     10
Length: 52, dtype: int64

In [54]:
# now draw a hand of 5 cards from the deck
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

AD     1
8C     8
5H     5
QC    10
2C     2
dtype: int64

In [55]:
# suppose you wanted 2 random cards from each suit
# because the suit is the last character of each card name, we can group based on this and use apply()

get_suit = lambda card : card[-1] # last letter is the suit
deck.groupby(get_suit).apply(draw, n=2) 

C  2C     2
   3C     3
D  QD    10
   8D     8
H  QH    10
   3H     3
S  2S     2
   4S     4
dtype: int64

In [56]:
# alternatively, we could write

deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

QC    10
JC    10
AD     1
5D     5
5H     5
6H     6
7S     7
QS    10
dtype: int64

## 10.3.5 Example: Group Weighted Average and Correlation

In [57]:
# under the split-apply-combine paradigm of groupby()
# operations between columns in a DataFrame or two Series, such as group weighted average, are possible

# create data
df = pd.DataFrame({'category' : ['a', 'a', 'a', 'a',
                                 'b', 'b', 'b', 'b'],
                   'data' : np.random.randn(8),
                   'weights' : np.random.rand(8)})

df

,category,data,weights
0,a,1.561587,0.957515
1,a,1.219984,0.347267
2,a,-0.482239,0.581362
3,a,0.315667,0.217091
4,b,-0.047852,0.894406
5,b,-0.454145,0.918564
6,b,-0.556774,0.277825
7,b,0.253321,0.955905


In [58]:
# the group weighted average by 'category' would then be
grouped = df.groupby('category')
get_wavg = lambda g : np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.811643
b   -0.122262
dtype: float64

In [59]:
# as another example, consider a financial dataset originally obtained from Yahoo! Finance 
# containing end-of-day prices for a few stocks and the S&P 500 index (the SPX symbol):
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [60]:
close_px

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
...,...,...,...,...
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66


In [61]:
# you might want to compute a DataFrame consisting only the year correlations of daily returns (computed from % changes) with SPX
# here's one way to do it
# first, create a function that computes the pairwise correlation of each column with the 'SPX' column
spx_corr = lambda x : x.corrwith(x['SPX'])

# next, compute the percentage change on close_px using pct_change()
rets = close_px.pct_change().dropna()

# lastly, group these percentage changes by year
# which can be extract from each row label with a lambda function that returns the 'year' attribute of each datetime label
get_year = lambda x : x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [62]:
# you could also compute inter-column correlations
# here we compute the annual correlation between Apple and Microsoft
by_year.apply(lambda g : g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

## 10.3.6 Example: Group-Wise Linear Regression

In [63]:
# you can use groupby() to perform the complex group-wise statistical analysis
# as long as the function returns a pandas object or scalar value

# example, i can define the following regress() function (using 'statmodels econometrics' library)
# which executes an ordinary least squares (OLS) regression on each chunk of data
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X =data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 Pivot Tables and Cross-Tabulation

In [64]:
# a 'pivot table' is a data summarization tool frequently found in spreadsheet programs and other data analysis software
# it aggregates a table of data by one or more keys, 
# arranging the data in a rectangle with some of the group keys along the rows and some along the columns
# pivot tables in Python with pandas are made possible through the grouby() facility combine with reshape operations utilizing hierarchical indexing

# DataFrame has a pivot_table() method, and there is also a top level function pandas.pivot_table()
# in addition, to provide a convenience interface to groupby()
# pivot_table() can add partial totals, also known as 'margins'

# returning to the tipping dataset
# suppose you wanted to compute a table of group means arranged by day and smoker on the rows
tips.pivot_table(index=['day', 'smoker']) # the default aggregation type of pivot_table() is mean

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [68]:
# this could have been produced with groupby() directly 
tips.groupby(['day', 'smoker'])[['size', 'tip', 'tip_pct', 'total_bill']].apply('mean')

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [69]:
# supposed we want to aggregate only tip_pct and size, and additionally group by time 
# i'll put smoker in the table columns and day in the rows
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [70]:
# we could augment this table to include partial totals by passing margins=True
# this has the effect of adding ALL row and column labels
# with corresponding values being the group statistics for all the data within a single tier

tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [71]:
# to use a different aggregation function, pass it to 'aggfunc'
# for example, 'count' or 'len' will give you a cross-tabulation (count or frequency) of group sizes

tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', 
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [72]:
# if some combinations are empty (or otherwise NA), you may wish to pass a 'fill_value'

tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
...                      ...       ...       ...       ...
Lunch  3    Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

[21 rows x 4 columns]

## 10.4.1 Cross-Tabulation: Crosstab

In [74]:
# a cross-tabulation (or crosstab for short) is a special case of a pivot table that computes group frequencies
# here's an example

from io import StringIO

# created data

data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [75]:
# as part of some survey analysis, we might want to summarize this data by nationality and handedness
# you could use pivot_table() to do this, but the pandas.crosstab() function can be more convenient

pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [76]:
# the first two arguments to crosstab() can each either be an array or Series or a list of arrays
# as in the tips data

pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244